In [1]:
from typing import List, Optional
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)
        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: List[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

In [4]:
# Problem Category: Easy
# Link: https://leetcode.com/problems/find-mode-in-binary-search-tree/submissions/1187947888
class Solution:
    def findMode(self, root: BinaryTreeNode) -> List[int]:
        def backtrack(curr: BinaryTreeNode):
            if curr == None:
                return
            else:
                hm[curr.val] = hm.get(curr.val, 0) + 1
                backtrack(curr.left)
                backtrack(curr.right)

        hm: dict[int, int] = dict()
        backtrack(root)
        max_ = max(hm.values())
        result: list[int] = []
        for k, v in hm.items():
            if v == max_:
                result.append(k)
        return result

# Testing the solution
assert Solution().findMode(BinaryTreeNode.from_array([1,None,2,2])) == [2]
assert Solution().findMode(BinaryTreeNode.from_array([0])) == [0]

In [5]:
# Problem Category: Medium
# Link: https://leetcode.com/problems/find-bottom-left-tree-value/submissions/1188894879/
class Solution:
    def findBottomLeftValue(self, root: BinaryTreeNode) -> int:
        next_: List[BinaryTreeNode] = [root]
        while True:
            next_next_: List[BinaryTreeNode] = []
            for node in next_:
                if node.left:
                    next_next_.append(node.left)
                if node.right:
                    next_next_.append(node.right)
            if next_next_:
                next_ = next_next_
            else:
                break
        return next_[0].val

# Testing the solution
assert Solution().findBottomLeftValue(BinaryTreeNode.from_array([2, 1, 3])) == 1
assert Solution().findBottomLeftValue(BinaryTreeNode.from_array([1,2,3,4,None,5,6,None,None,7])) == 7

In [6]:
# Problem Category: Easy
# Link: https://leetcode.com/problems/maximum-odd-binary-number/1190823057
class Solution:
    def maximumOddBinaryNumber(self, s: str) -> str:
        counts = dict(collections.Counter(s))
        zeros = counts.get('0', 0) * '0'
        ones = (counts.get('1', 0) - 1) * '1'
        return ones + zeros + '1'

# Testing the solution
assert Solution().maximumOddBinaryNumber("010") == "001"
assert Solution().maximumOddBinaryNumber("0101") == "1001"

In [7]:
# Problem Category: Easy
# Link: https://leetcode.com/problems/squares-of-a-sorted-array
class Solution:
    def sortedSquares(self, nums: list[int]) -> list[int]:
        N = len(nums)
        i, j = 0, N - 1
        sorted_by_magnitude: list[int] = []
        while i <= j:
            if abs(nums[i]) > abs(nums[j]):
                sorted_by_magnitude.append(nums[i])
                i += 1
            else:
                sorted_by_magnitude.append(nums[j])
                j -= 1
        sorted_by_magnitude.reverse()

        result: list[int] = []
        for num in sorted_by_magnitude:
            result.append(num * num)

        return result

# Testing the solution
assert Solution().sortedSquares([-4,-1,0,3,10]) == [0, 1, 9, 16, 100]
assert Solution().sortedSquares([-7,-4,-1,0,3,10]) == [0, 1, 9, 16, 49, 100]

In [8]:
# Problem Category: Medium
# Link: https://leetcode.com/problems/minimum-length-of-string-after-deleting-similar-ends/submissions/1194814297
class Solution:
    def minimumLength(self, s: str) -> int:
        N = len(s)
        i, j = 0, N - 1
        while i < j:
            while i < N - 1 and s[i + 1] == s[i] and s[i] == s[j]:
                i += 1
            while j > 1 and s[j - 1] == s[j] and s[i] == s[j]:
                j -= 1
            if s[i] == s[j]:
                i += 1
                j -= 1
            else:
                return (j - i) + 1
        else:
            return 1 if i == j else 0

# Testing the solution
assert Solution().minimumLength("ca") == 2
assert Solution().minimumLength("cabaabac") == 0
assert Solution().minimumLength("aabccabba") == 3
assert Solution().minimumLength("bbbbbabbbbccbcbcbccbbabbb") == 1
assert Solution().minimumLength("aaaa") == 0

In [9]:
# Problem Category: Easy
# Link: https://leetcode.com/problems/ransom-note/submissions/1195815128
class Solution:
    def canConstruct(self, ransomNote: str, magazine: str) -> bool:
        cr = collections.Counter(ransomNote)
        cm = collections.Counter(magazine)
        for i in range(ord("a"), ord("z") + 1):
            ch = chr(i)
            if cm.get(ch, 0) < cr.get(ch, 0):
                return False
        else:
            return True

# Testing the solution
assert Solution().canConstruct("aa", "aab") == True
assert Solution().canConstruct("a", "b") == False
assert Solution().canConstruct("aa", "ab") == False

Striver: Pascal's triangle
Problem: Finding nCr in minimal time (part of pascal's triangle)
Find element at position 1st 2nd column in a pascal's triangle
Sol: math.comb(n -1, r - 1)

In [10]:
def nCr(n: int, r: int) -> float:
    num = functools.reduce(lambda x, y: x * y, range(n, n - r, -1))
    den = functools.reduce(lambda x, y: x * y, range(r, 0, -1))
    return num / den

# Testing the solution
for i in range(50):
    i, j = sorted([random.randint(1, 50), random.randint(1, 50)], reverse=True)
    assert math.comb(i, j) == nCr(i, j), f"failed for {i, j}"

In [11]:
# 4Sum
class Solution:
    def fourSum(self, arr: list[int], target: int) -> set[tuple[int]]:
        def update_until(idx: int, update: int, until: int):
            temp_idx = idx
            assert update != 0
            if update > 0:
                while temp_idx < until and arr[temp_idx] == arr[idx]:
                    temp_idx += update
            else:
                while temp_idx > until and arr[temp_idx] == arr[idx]:
                    temp_idx += update
            return temp_idx

        N = len(arr)
        arr.sort()
        result: set[tuple[int]] = set()
        i = 0
        while i < N - 3:
            j = i + 1
            while j < N - 2:
                k, l = j + 1, N - 1
                while k < l:
                    curr = arr[i], arr[j], arr[k], arr[l]
                    if sum(curr) == target:
                        result.add(curr)
                        # Increment k, l
                        k_, l_ = k, l
                        while k_ < l_ and arr[k_] == arr[k] and arr[l_] == arr[l]:
                            k_, l_ = k_ + 1, l_ - 1
                        k, l = k_, l_
                    elif sum(curr) < target:
                        # Increment k
                        k = update_until(k, 1, l)
                    else:
                        # Decrement l
                        l = update_until(l, -1, k)

                # Increment j
                j = update_until(j, 1, N - 2)

            # Increment i
            i = update_until(i, 1, N - 3)

        return result

# Testing the solution
assert Solution().fourSum([1,0,-1,0,-2,2], 0) == {(-2,-1,1,2), (-2,0,0,2), (-1,0,0,1)}
assert Solution().fourSum([2,2,2,2,2], 8) == {(2, 2, 2, 2)}

Leetcode Weekly contest - 17th March

In [12]:
class Solution:
    def isSubstringPresent(self, s: str) -> bool:
        rev_ = "".join(list(reversed(s)))
        N = len(s)
        for i in range(N - 1):
            if s[i:i+2] in rev_:
                return True
        return False

# Testing the solution
assert Solution().isSubstringPresent("leetcode") == True
assert Solution().isSubstringPresent("abcd") == False

zz - z, z, zz (3)
zzz - z, z, z, zz, zz, zzz (6)
zzzz - z, z, z, z; zz, zz, zz, zzz, zzz, zzzz (10)
zzzzz - z, z, z, z, z, zz, zz, zz, zz, zzz, zzz, zzz, zzzz, zzzz, zzzzz (15)

In [13]:
class Solution:
    def countSubstrings(self, s: str, c: str) -> int:
        count = 0
        for ch in s:
            if ch == c:
                count += 1
        return (count * (count + 1)) / 2

# Testing the solution
assert Solution().countSubstrings("abada", "a") == 6
assert Solution().countSubstrings("zzz", "z") == 6

In [14]:
class Solution:
    def minimumDeletions(self, word: str, k: int) -> int:
        freq: list[int] = sorted(collections.Counter(word).values())
        N = len(freq)

        result = math.inf
        sum_ = 0
        for i in range(N):
            sum_ += freq[i - 1] if i > 0 else 0
            curr_ = 0
            for j in range(i + 1, N):
                if freq[j] - freq[i] > k:
                    curr_ += freq[j] - freq[i] - k
            result = min(curr_ + sum_, result)

        return result

# Testing the solution
assert Solution().minimumDeletions("aabcaba", 0) == 3
assert Solution().minimumDeletions("dabdcbdcdcd", 2) == 2
assert Solution().minimumDeletions("aaabaaa", 2) == 1

Leetcode Weekly contest - 17th March
Link: https://leetcode.com/problems/search-in-rotated-sorted-array/submissions/1210464524/

In [15]:
class Solution:
    def search(self, nums: List[int], target: int) -> int:
        N = len(nums)
        low, high = 0, N - 1
        while low <= high:
            mid = (low + high) // 2
            if nums[mid] == target:
                return mid

            # Left portion is sorted
            elif nums[low] <= nums[mid]:
                if nums[low] <= target <= nums[mid]:
                    high = mid - 1
                else:
                    low = mid + 1

            # Right portion is sorted
            else:
                if nums[mid] <= target <= nums[high]:
                    low = mid + 1
                else:
                    high = mid - 1

        # Not found
        else:
            return -1

# Testing the solution
assert Solution().search([4,5,6,7,0,1,2], 0) == 4
assert Solution().search([4,5,6,7,0,1,2], 3) == -1
assert Solution().search([1], 0) == -1